# **Hugging Face Backup**
<small> An extremely weird fork of content from Nocrypt's notebook, and as well as EveryDream2Trainer for JupyterLabs by a DID system that has no clue how to code. We'd call it opinionated! This is MISSING google stuff, we want to MAYBE add ocalmfuse to it eventually, it is NOW missing EMJOY finder because it's useless in a jupyter environment. (It is! LOL you can easy find stuff otherwise!)

*Quickly upload stuff like merged models to huggingface*
-----------------


-----------------
## ♦♦ **JUPYTER NOTEBOOK UPDATES** ♦♦
    
-----------------
    
If you've copied this to drive and it's no longer working check here: https://github.com/kieranxsomer/HuggingFace_Backup
If you're on CivitAI, the version SHOULD be updated as soon as a version has been pushed :3 

-----------------

##  **HUGGING FACE HUB BACKUP** 


At the moment this only supports CHECKPOINTS, rather than full backups. This works on a local machine.
FOLDERS may only work in theory - for MODEL types (as in diffusers adn otherwise) rather than anything else.
VAST USERS: THIS ONLY WORKS ON JUPYTER LAB, NOT SERVER. BE AWARE THAT RUNPOD USES LAB BY DEFAULT.

### ☢☢ **MAKE SURE:** ☢☢

You create a huggingface token, go to [this link](https://huggingface.co/settings/tokens), then `create new token` or copy available token with the `Write` role.

-----------------
### ***Forked from - EVERYDREAM2 TRAINER - LINAQRUF - AND NOCRYPT - Literally just yoinked it from this notebook:***
    
**https://colab.research.google.com/drive/1wEa-tS10h4LlDykd87TF5zzpXIIQoCmq** 
    
**https://github.com/victorchall/EveryDream2trainer**

**https://colab.research.google.com/github/Linaqruf/sdxl-model-converter/blob/main/sdxl_model_converter.ipynb**

-----------------

## Support? 

Earth and Dusk Discord because REASONS: https://discord.gg/5t2kYxt7An

Support my stupid lack of coding here: https://ko-fi.com/duskfallcrew

-----------------


CHANGES SINCE COLAB VERSION
    
    1. Yoinked 90% of the code from EveryDream2Trainer
    2. Keeping Credit for Nocrypt because that's who made the original stuff.
    3. ATTEMPTING to get the requirements IN The notebook in case it legit cries
    4. Crawed into a hole nobody can find me now - *hiding under a rock*








# Install Requirements

In theory this is collecting all the requirements, if i'm missing any then wait for an update or head into terminal and patch it yourself. (That's not sarcasm, i'm slow to fixing things, i'm a patchter not a programmer!) 

In [ ]:
!pip install glob2
!pip install --force-reinstall -qqqq huggingface_hub
!pip install transformers
!pip install ipywidgets

# HuggingFace Upload/Download (Optional)
Run the cell below and paste your token into the prompt.  You can get your token from your [huggingface account page](https://huggingface.co/settings/tokens).

The token will not show on the screen, just press enter after you paste it.

### Make sure to run this login cell for any Huggingface uploads or private downloads.


In [ ]:
from huggingface_hub import notebook_login, hf_hub_download
import os
notebook_login()

## Upload files Repository Setup & Diffusers Upload

This is yoinked from Linaqruf's SDXL conversion for diffusers. I don't know if it works FOR anything but diffusers, but it should?  - This one you put in your write token

In [ ]:
# @title ### **Huggingface Hub config**
from huggingface_hub import login, HfApi
from huggingface_hub.utils import validate_repo_id, HfHubHTTPError

# @markdown Login to Huggingface Hub
# @markdown > Get **your** huggingface `WRITE` token [here](https://huggingface.co/settings/tokens)
write_token = "TOKEN HERE"  # @param {type:"string"}
# @markdown Fill this if you want to upload to your organization, or just leave it empty.
orgs_name = ""  # @param{type:"string"}
# @markdown If your model repo does not exist, it will automatically create it.
model_name = "MODEL NAME"  # @param {type:"string"}
make_private = False  # @param{type:"boolean"}

def authenticate(write_token):
    login(write_token, add_to_git_credential=True)
    api = HfApi()
    return api.whoami(write_token), api

def create_model_repo(api, user, orgs_name, model_name, make_private=False):
    if orgs_name == "":
        repo_id = user["name"] + "/" + model_name.strip()
    else:
        repo_id = orgs_name + "/" + model_name.strip()

    try:
        validate_repo_id(repo_id)
        api.create_repo(repo_id=repo_id, repo_type="model", private=make_private)
        print(f"Model repo '{repo_id}' didn't exist, creating repo")
    except HfHubHTTPError as e:
        print(f"Model repo '{repo_id}' exists, skipping create repo")

    print(f"Model repo '{repo_id}' link: https://huggingface.co/{repo_id}\n")

    return repo_id

user, api = authenticate(write_token)

if model_name:
    model_repo = create_model_repo(api, user, orgs_name, model_name, make_private)


In [ ]:
# @title ### **Upload to Huggingface**
from huggingface_hub import HfApi
from pathlib import Path
import os

api = HfApi()

# @markdown This will be uploaded to model repo
model_path = "MODEL PATH"  # @param {type :"string"}
path_in_repo = "/tree/main"  # @param {type :"string"}
project_name = os.path.basename(model_path)

# @markdown Other Information
commit_message = ""  # @param {type :"string"}

def is_diffusers_model(model_path):
    required_folders = {"unet", "text_encoder", "text_encoder_2", "tokenizer", "tokenizer_2", "scheduler", "vae"}
    return required_folders.issubset(set(os.listdir(model_path))) and os.path.isfile(os.path.join(model_path, "model_index.json"))

def upload_model(model_paths, is_folder: bool):
    path_obj = Path(model_paths)
    trained_model = path_obj.parts[-1]

    path_in_repo_local = path_in_repo if path_in_repo and not is_diffusers_model(model_paths) else ""

    notification = f"Uploading {trained_model} from {model_paths} to https://huggingface.co/{model_repo}"
    print(notification)

    if is_folder:
        if is_diffusers_model(model_paths):
            commit_message = f"Upload diffusers format: {trained_model}"
            print("Detected diffusers model. Adjusting upload parameters.")
        else:
            commit_message = f"Upload checkpoint: {trained_model}"
            print("Detected regular model. Adjusting upload parameters.")

        api.upload_folder(
            folder_path=model_paths,
            path_in_repo=path_in_repo_local,
            repo_id=model_repo,
            commit_message=commit_message,
            ignore_patterns=".ipynb_checkpoints",
        )
    else:
        commit_message = f"Upload file: {trained_model}"
        api.upload_file(
            path_or_fileobj=model_paths,
            path_in_repo=path_in_repo_local,
            repo_id=model_repo,
            commit_message=commit_message,
        )

    success_notification = f"Upload successful. Check the model at https://huggingface.co/{model_repo}/tree/main"
    print(success_notification)

def upload():
    if model_path.endswith((".ckpt", ".safetensors", ".pt")):
        upload_model(model_path, False)
    else:
        upload_model(model_path, True)

upload()


## Upload files from a folder (Single files)

You'll need to note in "REPO" where you're uploading to - please go TO your model page or sorry your user page, create a new repo.  This means that unlike colab, where it can CREATE ONE - somehow on jupyter lab it's easier to just have one pre created. This is for MULTIPLE FILES at a time, but please note it won't do folders.

THIS SHOULD give you a choice to Upload ALL OF THE FIILESS!

In [ ]:
import glob
import os
from huggingface_hub import HfApi
from ipywidgets import *

all_ckpts = [f for f in glob.glob("*.safetensors")]
  
ckpt_picker = SelectMultiple(options=all_ckpts, layout=Layout(width="600px")) 
hfuser = Text(placeholder='Y0URUS3RH3R3')
hfrepo = Text(placeholder='THEMODELSPACE')

api = HfApi()
upload_btn = Button(description='Upload')
out = Output()

def upload_ckpts(_):
    repo_id=f"{hfuser.value or hfuser.placeholder}/{hfrepo.value or hfrepo.placeholder}"
    with out:
        if ckpt_picker is None or len(ckpt_picker.value) < 1:
            print("Nothing selected for upload, make sure to click one of the ckpt files in the list, or, you have no ckpt files in the current directory.")
        for ckpt in ckpt_picker.value:
            print(f"Uploading to HF: huggingface.co/{repo_id}/{ckpt}")
            response = api.upload_file(
                path_or_fileobj=ckpt,
                path_in_repo=ckpt,
                repo_id=repo_id,
                repo_type=None,
                create_pr=1,
                commit_message = "Upload with 🤗 Earth & Dusk's Amazing HF Backup for Vast & Runpod"
            )
            display(response)
        print("DONE")
        print("Go to your repo and accept the PRs this created to see your files")

upload_btn.on_click(upload_ckpts)
box = VBox([ckpt_picker, HBox([hfuser, hfrepo]), upload_btn, out])

display(box)